In [1]:
import pickle
import numpy as np
import pandas as pd
import vectorbt as vbt
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
from sklearn.metrics import make_scorer
import warnings
import matplotlib.pyplot as plt
import itertools
from datetime import datetime, timedelta
import pytz
from numba import njit
import ipywidgets

import warnings
warnings.filterwarnings('ignore')

In [ ]:
fixed_dollar_amount = 1000

# Price to use for entry sizing (use Open to avoid lookahead)
entry_price = ohlcv['Open']

# Create size DataFrame matching entries/exits
size = pd.DataFrame(0.0, index=entries.index, columns=entries.columns)

# Fill only where entries are True: size = $1000 / entry_price
size[entries] = fixed_dollar_amount / entry_price[entries]

# Optional: round to 4 decimals or fewer for asset precision
size = size.round(4)

# --- Build portfolio ---
portfolio = vbt.Portfolio.from_signals(
    close=ohlcv['Close'],
    entries=entries,
    exits=exits,
    size=size,
    accumulate=True,
    direction='longonly',
    init_cash=10_000
)

# --- Review ---
print(portfolio.stats())
portfolio.plot().show()

In [5]:
seed = 42
symbols = [
    'BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'INJUSDT', 'NEARUSDT', 
    'ENSUSDT', 'AAVEUSDT', 'UMAUSDT','BCHUSDT', 'LTCUSDT',
    'LINKUSDT', 'LDOUSDT'
]
start_date = datetime(2022, 7, 5, tzinfo=pytz.utc)
end_date = datetime(2025, 7, 5, tzinfo=pytz.utc)
time_delta = end_date - start_date
window_len = timedelta(days=180)
window_count = 400
exit_types = ['TP', 'Holding']
step = 0.01  # 1%
stops = np.arange(step, 1 + step, step)

vbt.settings.array_wrapper['freq'] = 'd'
vbt.settings.plotting['layout']['template'] = 'vbt_dark'
vbt.settings.portfolio['init_cash'] = 100.  # 100$
vbt.settings.portfolio['fees'] = 0.0025  # 0.25%
vbt.settings.portfolio['slippage'] = 0.0025  # 0.25%

print(pd.Series({
    'Start date': start_date,
    'End date': end_date,
    'Time period (days)': time_delta.days,
    'Assets': len(symbols),
    'Window length': window_len,
    'Windows': window_count,
    'Exit types': len(exit_types),
    'Stop values': len(stops),
    'Tests per asset': window_count * len(stops) * len(exit_types),
    'Tests per window': len(symbols) * len(stops) * len(exit_types),
    'Tests per exit type': len(symbols) * window_count * len(stops),
    'Tests per stop type and value': len(symbols) * window_count,
    'Tests total': len(symbols) * window_count * len(stops) * len(exit_types)
}))

ModuleNotFoundError: No module named 'yfinance'